In [19]:
import torch
import math

torch.set_grad_enabled(False)

In [20]:
class Module(object) :
    
    def  forward(self , *input):
        raise  NotImplementedError
        
    def  backward(self , *gradwrtoutput):
        raise  NotImplementedError
        
    def  param(self):
        return  []

    

In [21]:
def tanh(x):
    return math.tanh(x)

In [22]:
def dtanh(x):
    return 1 + tanh(x)**2

In [23]:
def MSEloss(data_input, data_target):
    x = torch.pow(data_input - data_target, 2)
    x = x.sum()
    return x / data_input.size(0)